# Viabilidade do S.A. para problemas com grafos

Esse documento estuda a viabilidade do S.A. para problemas com grafos.

O problema principal é a definição da variável de estado. A variável de estado deve ser tal que aceite uma pertubação aleatória com distribuição gaussiana ou de cauchy e que permita que essa pertubação, na maioria das vezes, seja tal que o próximo estado permaneça no conjunto viável.

Para executar esse notebook, são necessárias as seguintes bibliotecas:

In [111]:
#Padrão
import time

# 3th party
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Definição do grafo

O código abaixo gera um grafo com `numeroNos` nós e uma razão `razaoEnlacesPorNo`. Para isso, são definidas as funções:


In [67]:
def shuffle_along_axis(a, axis):
    """
    Embaralha a ordem dos valores de uma matriz em uma dimensão. O embaralhamento é diferente para cada elemento da dimensão da matriz.
    """
    idx = np.random.rand(*a.shape).argsort(axis=axis)
    return np.take_along_axis(a,idx,axis=axis)

def add_zero_valued_diagonal(a):
    """
    Adiciona uma diagonal de valor nulo a uma matriz
    """
    teste = np.concatenate([np.zeros((1,a.shape[1])),a])
    for i in range(teste.shape[0]):
        teste[:,i] = np.roll(teste[:,i],i)
    return teste

In [137]:
def CriarMatrizCustoGrafo(numeroNos,proporcaoEnlaces,simetrico=False,deterministico=False,seed=1):
    
    # Trava os resultados aleatorios para geração da matriz
    if deterministico:
        np.random.seed(seed)
    
    # Gera uma matriz de custo considerando um grafo completamente conectado
    C = np.random.randint(1,20,size=(numeroNos,numeroNos))
    
    # Gera um filtro para a matriz de custo que depende do número de enlaces
    filtroEnlaces = np.concatenate([np.zeros((proporcaoEnlaces,numeroNos)),np.ones((numeroNos-proporcaoEnlaces-1,numeroNos))])
    filtroEnlaces = shuffle_along_axis(filtroEnlaces,0)*1e20
    filtroEnlaces = add_zero_valued_diagonal(filtroEnlaces)
    filtroEnlaces
    
    # Aplica o filtro a função custo
    C = (C+filtroEnlaces)*(1-np.eye(numeroNos))
    
    # Transforma a matriz em uma matriz simétrica para o caso de grafos não direcionados
    if simetrico == True:
        C = np.triu(C) + np.triu(C).T
    
    # Reinicia o gerador em um valor qualquer
    if deterministico:
        np.random.seed(np.floor(time.time()*10**6))
    
    return C

A matriz de custo tem as seguintes características:
* $c_{ii}=0\forall i \in I={1,\dots,N}$
* $c_{ij}\forall i\neq j$ tem o custo normal (O.G. 10^1) para o caso da matriz possuir um enlace entre i e j
* $c_{ij}\forall i\neq j$ tem o custo elevadíssimo (O.G. 10^20) para o caso da matriz não possuir um enlace entre i e j



# Definição do domínio

Para um grafo com $N$ vértices e $M$ arestas, vamos definir um vetor de fluxos nas arestas $\vec{a}\in\mathbb{R}^M$. Vamos supor que o grafo é densamente conectado, ou seja, $M\gg N$. Escrevemos um sistema sub-determinado da seguinte forma:
$$\begin{bmatrix}F_{0}\\ F_{c}\end{bmatrix}\vec{a}=\begin{bmatrix}\vec{0}\\ \vec{c}\end{bmatrix}$$

A matriz $F_{0}$ representa a conservação de fluxo para os nós intermediários, ou seja, aqueles que não são nem fonte e nem destino de um grafo. Já a matriz $F_c$ representa a conservação de fluxo para nós que são origem ou destino, onde para cada linha de $F_c$, o valor do elemento em $\vec{c}$ correspondente é
$$c_i=\begin{cases}+f_i\text{, soma dos fluxos que saem de }i\\ -f_i\text{, soma dos fluxos que entram em }i\end{cases}$$

Cada linha de $F_0$ define um nó na qual a soma dos fluxos dos vértices entrando no nó devem se igualar a soma dos vértices saindo do nó. Portanto o número de linhas de $F_0$ é igual ao número de nós intermediários. Já o número de linhas de $F_c$ será igual ao número de vértices que são ou fonte ou destino de algum contrato. Assim, sabemos que a matriz $\begin{bmatrix}F_{0}\\ F_{c}\end{bmatrix}$ sempre será de ordem $\{0,1\}^{N\times M}$.

Objetivo: Quero encontrar (ou definir) $\vec{a}_i$ arestas independentes para que eu possa alterar o valor livremente e definir uma transformação linear $T$ na qual $T\vec{a}_i=\vec{a}$.


# Devaneios

* Vamos manter apenas a conservação de fluxo como restrição e passar as condições de contrato para a função custo. É possível definir na função de custo uma distância entre o fluxo determinado dos contratos e o fluxo atual.
* 